<a href="https://colab.research.google.com/github/andresdp/things-blockchain/blob/main/pinta1_geers2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **GEERS 2022 - Pinta #1**

---
### **Hashing**

In [2]:
from hashlib import sha256

s = input("Type anything:")
hash = sha256(s.encode('utf-8')).hexdigest()
print("The hash for:",s,"is:", hash)

Type anything:test
The hash for: test is: 9f86d081884c7d659a2feaa0c55ad015a3bf4f1b2b0b822cd15d6c15b0f00a08


---
### **Proof of Work**



In [13]:
import random
from timeit import default_timer as timer
  
# It implements the proof of work for a given input
# The difficulty level is taken as a configuration parameter
def proof_of_work(input, difficulty=3):
  print("Doing proof of work for:",input, "( difficulty:",difficulty,")")
  start = timer()
  header = '0'*difficulty
  nonce = random.random()
  #print(header)
  while True:
    nonce += 1
    s = input + str(nonce)
    guess_hash = sha256(s.encode('utf-8')).hexdigest()
    #print(guess_hash)
    if guess_hash[:difficulty] == header:
      print("  ", guess_hash)
      end = timer()
      print("  ", "elapsed time (secs):", end - start) # Time in seconds
      return nonce # The puzzle was solved!

In [14]:
my_input = "Geers is awesome"
nonce = proof_of_work(my_input)
nonce

Doing proof of work for: Geers is awesome ( difficulty: 3 )
   000961b9b3828c7794bba92041a7444b19c0f919fe6ee220e9d55d6abee7e79c
   elapsed time (secs): 0.005060113000126876


2223.3332041789668

In [15]:
# Verification
s = my_input + str(nonce)
sha256(s.encode('utf-8')).hexdigest()

'000961b9b3828c7794bba92041a7444b19c0f919fe6ee220e9d55d6abee7e79c'

In [16]:
proof_of_work("Geers is awesome", difficulty=6)

Doing proof of work for: Geers is awesome ( difficulty: 6 )
   000000cc9745921e449f20b0e2b1cfdc3c5c5453f65d91ec41c5d6590666d905
   elapsed time (secs): 121.52318882999998


57758829.13051548

---
### **Arbol de Merkle**

In [4]:
!pip install pymerkle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49 kB 5.1 MB/s 


In [37]:
from pymerkle import MerkleTree

tree = MerkleTree()

# Populate tree with some records
records = [b'foo', b'bar', b'baz', b'qux', b'quux']
for data in records:
    tree.encrypt(data)
print(tree)
tree.export('backup.json')


 └─a2df9c3b0ca85b3dec24ebb128a05488ed6f20925faf67d1ed7f97025fdbf58c
     ├──2552e3cbf21513bf4b7a8134e4e4a2e1cd8857cc7fe8a51a455ec226e5cf1e6c
     │    ├──79baf30a8e17b9538bbb00dc43a3ef9471909f5729f52cb5b13bdacab788feae
     │    │    ├──1d2039fa7971f4bf01a1c20cb2a3fe7af46865ca9cd9b840c2063df8fec4ff75
     │    │    └──485904129bdda5d1b5fbc6bc4a82959ecfb9042db44dc08fe87e360b0a3f2501
     │    └──7c674ef08ee554408515fb2c0d4bb5833b3b93fda37dfaaa4db09f929cf34d65
     │         ├──b06d695869f105fffa5f68c4b9628d58a1aff469a3c62c8c74ddb2af47b178ef
     │         └──5a43bc145b8c045b535615392edfba2bd2c4556dfdb13f968bd0668c042ecb35
     └──2e219794e81e9bb22e230e7f40906eecb4cfcc289e1af253d88b23634266c5a2



In [47]:
tree.get_config()

{'encoding': 'utf_8', 'hash_type': 'sha256', 'security': True}

In [38]:
state = tree.get_root_hash()
state

b'a2df9c3b0ca85b3dec24ebb128a05488ed6f20925faf67d1ed7f97025fdbf58c'

In [39]:
from pymerkle.hashing import HashEngine

challenge = HashEngine(**tree.get_config()).hash('qux') # Takes a dict and extracts its contents
challenge

b'5a43bc145b8c045b535615392edfba2bd2c4556dfdb13f968bd0668c042ecb35'

In [41]:
# Prove and verify encryption of data (i.e., if the challenge is actually part of the tree)
proof1 = tree.generate_audit_proof(challenge)
proof1.verify()

True

In [42]:
proof1


    ----------------------------------- PROOF ------------------------------------

    uuid        : dec939e2-11d6-11ed-bd91-0242ac1c0002

    timestamp   : 1659385007 (Mon Aug  1 20:16:47 2022)
    provider    : d1ad8bd2-11d6-11ed-bd91-0242ac1c0002

    hash-type   : SHA256
    encoding    : UTF-8
    security    : ACTIVATED

    
       [0]   +1   79baf30a8e17b9538bbb00dc43a3ef9471909f5729f52cb5b13bdacab788feae
       [1]   -1   b06d695869f105fffa5f68c4b9628d58a1aff469a3c62c8c74ddb2af47b178ef
       [2]   -1   5a43bc145b8c045b535615392edfba2bd2c4556dfdb13f968bd0668c042ecb35
       [3]   -1   2e219794e81e9bb22e230e7f40906eecb4cfcc289e1af253d88b23634266c5a2

    offset      : 2

    commitment  : a2df9c3b0ca85b3dec24ebb128a05488ed6f20925faf67d1ed7f97025fdbf58c

    -------------------------------- END OF PROOF --------------------------------

In [43]:
# Save current tree state
state = tree.get_root_hash()

# Append further leaves
for data in [b'corge', b'grault', b'garlpy']:
    tree.encrypt(data)

In [44]:
print(tree)


 └─e45204e846a110051b8848afa80d195317c25554358c5d4e2e1363993dd8537b
     ├──2552e3cbf21513bf4b7a8134e4e4a2e1cd8857cc7fe8a51a455ec226e5cf1e6c
     │    ├──79baf30a8e17b9538bbb00dc43a3ef9471909f5729f52cb5b13bdacab788feae
     │    │    ├──1d2039fa7971f4bf01a1c20cb2a3fe7af46865ca9cd9b840c2063df8fec4ff75
     │    │    └──485904129bdda5d1b5fbc6bc4a82959ecfb9042db44dc08fe87e360b0a3f2501
     │    └──7c674ef08ee554408515fb2c0d4bb5833b3b93fda37dfaaa4db09f929cf34d65
     │         ├──b06d695869f105fffa5f68c4b9628d58a1aff469a3c62c8c74ddb2af47b178ef
     │         └──5a43bc145b8c045b535615392edfba2bd2c4556dfdb13f968bd0668c042ecb35
     └──3125448f11f80f8f0a8d4523d5d7d67ecf74bd54f8545a868e4424821fc6c94d
          ├──a6f25850daba7cc16e2d0b4f1484459f478361db11546199e0304476b4a0bf6d
               ├──2e219794e81e9bb22e230e7f40906eecb4cfcc289e1af253d88b23634266c5a2
               └──1c0c3f2623f61894bb6ab743c75efe35632b67985777b7017dedcddc3720aae9
          └──5b80e74a146a8a28cf4623b7a2463112f2b98633

In [45]:
# Prove that the current state is an evolved version of the previous tree
proof2 = tree.generate_consistency_proof(challenge=state)
proof2.verify()

True

In [46]:
proof2


    ----------------------------------- PROOF ------------------------------------

    uuid        : 235bbc24-11d7-11ed-bd91-0242ac1c0002

    timestamp   : 1659385122 (Mon Aug  1 20:18:42 2022)
    provider    : d1ad8bd2-11d6-11ed-bd91-0242ac1c0002

    hash-type   : SHA256
    encoding    : UTF-8
    security    : ACTIVATED

    
       [0]   +1   2552e3cbf21513bf4b7a8134e4e4a2e1cd8857cc7fe8a51a455ec226e5cf1e6c
       [1]   +1   2e219794e81e9bb22e230e7f40906eecb4cfcc289e1af253d88b23634266c5a2
       [2]   +1   1c0c3f2623f61894bb6ab743c75efe35632b67985777b7017dedcddc3720aae9
       [3]   -1   5b80e74a146a8a28cf4623b7a2463112f2b98633e4c11874e74117f55f8f291c

    offset      : 1

    commitment  : e45204e846a110051b8848afa80d195317c25554358c5d4e2e1363993dd8537b

    -------------------------------- END OF PROOF --------------------------------

---